# 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import urllib.request
import urllib

In [2]:
# 크롬드라이버 위치
path = r"C:/Users/NT551XCJ/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"

In [38]:
# 크롤링할 url
url = 'https://www.kobis.or.kr/kobis/business/stat/offc/searchOfficHitTotList.do?searchMode=year'

s = Service(path)
driver = webdriver.Chrome(service=s)

driver.get(url)

# 연도 설정 2010 클릭
btn_ex_path = '//*[@id="standYySt"]/option[40]'
driver.find_element(By.XPATH,btn_ex_path ).click()
time.sleep(1)

# 연도 설정 2021 클릭
btn_ex_path = '//*[@id="standYyEd"]/option[51]'
driver.find_element(By.XPATH,btn_ex_path ).click()
time.sleep(1)

# 검색 버튼 클릭
btn_ex_path2 = '//*[@id="searchForm"]/div/div[8]/button'
driver.find_element(By.XPATH,btn_ex_path2).click()
time.sleep(3)


In [39]:
# 크롤링 하기

n=1
result = []
# 7 * 10 -> 70페이지 까지만 크롤링
for k in range(3):
    # 페이지 10개
    for j in range(1,11):
        # 다음 페이지 버튼
        btn_ex_path5 = f'//*[@id="pagingForm"]/div/ul/li[{j}]/a'
        driver.find_element(By.XPATH,btn_ex_path5).click()
        time.sleep(3)
        # 리스트 10개
        for i in range(1,11):
            try:
                try:
                    # 영화 이름
                    name_path = f'//*[@id="content"]/div[4]/div[2]/table/tbody/tr[{i}]/td[2]/span/a'
                    name = driver.find_element(By.XPATH, name_path).text
                    img_name = re.sub(r'[\\/*?:"<>|]', '-', name)
                except:
                    name = ''
                    img_name = ''

                # 영화 클릭
                btn_ex_path3 = f'//*[@id="content"]/div[4]/div[2]/table/tbody/tr[{i}]/td[2]/span/a'
                driver.find_element(By.XPATH,btn_ex_path3).click()
                time.sleep(3)

                soup = BeautifulSoup( driver.page_source, "html.parser")

                try:
                    # 영화 이미지 링크 및 저장
                    img = 'https://www.kobis.or.kr' + soup.find_all("a", class_ = "fl thumb")[0].get('href')
                    img_name = re.sub(r'[\\/*?:"<>|]', '-', name)
                    urllib.request.urlretrieve(img, img_name + '.jpg')
                except:
                    img_name = ''

                try:
                    # 영화 코드
                    Mcode_path = f'//*[@id="ui-id-{n}"]/div/div[1]/div[2]/dl/dd[1]'
                    Mcode = driver.find_element(By.XPATH, Mcode_path).text
                except:
                    Mcode = ''

                try:
                    # 영화 정보
                    Minfo_path = f'//*[@id="ui-id-{n}"]/div/div[1]/div[2]/dl/dd[4]'
                    Minfo = driver.find_element(By.XPATH, Minfo_path).text
                except:
                    Minfo = ''

                # 상영타입
                try:
                    Mtype = []
                    for t in range(len(soup.find("dl", class_ = "ovf cont").find_all('strong'))):
                        Mtype.append(re.sub(r'[:]','',(soup.find("dl", class_ = "ovf cont").find_all('strong')[t].text)).strip())
                    Mtype = ','.join(Mtype)
                except:
                    Mtype = ''
                    pass

                # 주연
                try:
                    act = soup.select("dl > div:nth-child(2) > dd > table:nth-child(1) > tbody > tr > td")[0].text
                except:
                    act = ''
                    pass

                result.append([name, img, img_name + '.jpg', Mcode, Minfo, Mtype, act])
                n += 2

                # 닫기
                btn_ex_path4 = '/html/body/div[3]/div[1]/div[1]/a[2]/span'
                driver.find_element(By.XPATH,btn_ex_path4).click()
                time.sleep(3)

            except:
                # 오류난 위치 출력
                print(k,j,i)
                pass

    # 화살표 클릭
    btn_ex_path6 = '//*[@id="pagingForm"]/div/a[3]'
    driver.find_element(By.XPATH,btn_ex_path6).click()
    time.sleep(3)

print('Done')

# 데이터 프레임 저장
result_df = pd.DataFrame(result,columns = ['영화 이름', '이미지 링크', '이미지 이름', '영화 코드', '영화 정보', '상영타입', '주연배우'])

# csv 저장
result_df.to_csv("영화 크롤링.csv",index = True, encoding = 'utf-8')

Done


# 전처리
   | 로 구분 되어있던 영화 정보 컬럼을 여러 컬럼으로 분할

In [ ]:
# csv 불러오기
path = "C:/Users/NT551XCJ/Desktop/MLproject/영화 크롤링.csv"
df = pd.read_csv(path, encoding = 'cp949')

In [ ]:
df['길이'] = df['영화 정보'].apply(lambda x: x.split('|')[0].strip())
df['영화종류'] = df['영화 정보'].apply(lambda x: x.split('|')[1].strip())
df['장르2'] = df['영화 정보'].apply(lambda x: x.split('|')[2].strip())
df['러닝타임'] = df['영화 정보'].apply(lambda x: x.split('|')[3].strip())
df['권장연령'] = df['영화 정보'].apply(lambda x: x.split('|')[4].strip())

In [ ]:
df.to_csv("영화 크롤링.csv",index = False, encoding = 'cp949')

In [ ]:
# 조인

# csv 불러오기
path = "C:/Users/NT551XCJ/Desktop/MLproject/영화csv/영화 최종.csv"
df = pd.read_csv(path, encoding = 'utf-8')

# csv 불러오기
path1 = "C:/Users/NT551XCJ/Desktop/MLproject/영화csv/kobis 통합.csv"
df1 = pd.read_csv(path1, encoding = 'utf-8')

# 조인
temp_merge = pd.merge(left = df , right = df1, how = "left", on = ["영화 이름"])

# csv 저장
temp_merge.to_csv("영화 최종.csv",index = False, encoding = 'cp949')

In [ ]:
# 재개봉한 영화 보기
result = temp_merge.groupby('영화 이름').filter(lambda x: len(x) >= 2)
result

In [ ]:
# 재개봉한 영화 행 삭제 관객수가 가장 많았던 행만 남김
temp_merge = temp_merge.sort_values(by='전국 관객수', ascending=False)  # '전국 관객수'를 기준으로 내림차순 정렬합니다.
temp_merge = temp_merge.drop_duplicates(subset='영화 이름', keep='first')  # '영화 이름'이 중복되는 경우 첫 번째 행만 남깁니다.

# 조인

In [ ]:
# csv 불러오기 
path = "C:/Users/NT551XCJ/Desktop/MLproject/포스터 얼굴 분석.csv"
face_df = pd.read_csv(path, encoding = 'cp949')

# csv 불러오기 
path = "C:/Users/NT551XCJ/Desktop/MLproject/5colors_final.csv"
rgb_df = pd.read_csv(path, encoding = 'cp949')

# csv 불러오기 
path = "C:/Users/NT551XCJ/Desktop/MLproject/영화 최종.csv"
kobis_df = pd.read_csv(path, encoding = 'cp949')

# csv 불러오기 
path = "C:/Users/NT551XCJ/Desktop/MLproject/text.csv"
text_df = pd.read_csv(path, encoding = 'utf-8')

In [ ]:
# 각 길이가 맞는지 체크
len(face_df),len(rgb_df),len(kobis_df),len(text_df)

In [ ]:
# 이미지 이름을 기준으로 조인할 거기 때문에 이미지 이름으로 컬럼 명을 바꿔줌
rgb_df.rename(columns = {'Image_name':'이미지 이름'},inplace = True)

In [ ]:
# 4개의 데이터의 이미지 이름은 전부 같아야 함
# kobis데이터의 이미지 이름과 rgb,text,face의 이미지 이름을 전부 비교해서 다른 부분이 없는지 체크

# kobis_df에서 이미지 이름 추출
kobis_image_names = set(kobis_df['이미지 이름'].unique())

# rgb_df에서 타이틀 추출
rgb_titles = set(face_df['이미지 이름'].unique())

# kobis_df와 rgb_df 간의 이미지 이름과 타이틀 값이 다른 행 찾기
different_values = kobis_image_names.symmetric_difference(rgb_titles)

# 결과 출력
print(different_values)

In [ ]:
# 문제 없으면 전부 조인
temp_merge = pd.merge(left = kobis_rgb , right = rgb_df, how = "left", on = ["이미지 이름"])
csv_list = list(face_df,text_df)

for csv_name in csv_list:
    temp_merge = pd.merge(left = temp_merge , right = csv_name, how = "left", on = ["이미지 이름"])

temp_merge.to_csv("전체 데이터 원본.csv",index = False, encoding = 'cp949')

# 사용 안한 전처리 코드
배우 컬럼에 뭉텅이로 있던 것을 가로를 없애고 ,로 구분하게 만드는 코드

In [ ]:
# 장르 필터링 하기 (******최종적으로 사용 안함)
temp_merge = temp_merge[temp_merge['장르'].isin(['드라마', '액션', '코미디'])]

In [ ]:
# 배우 가로 없애기 (******최종적으로 사용 안함)
df = df[['영화 이름','주연배우', '전국 매출액', '전국 관객수']]
df['주연배우'] = df['주연배우'].apply(lambda x: ','.join([i.split('(')[0].strip() for i in re.sub(r'[|]', ',', re.sub('[주연]','',str(x))).split(',')]) if str(x) != '' else '')
df['주연배우'] = df['주연배우'].apply(lambda x: re.sub(r'[[]]','',x).strip() if str(x) != '' else '')

# csv 저장
df.to_csv("배우.csv",index = False, encoding = 'cp949')

# 정보 체크하는데 사용한 코드
조인 전에 조인할 키 값이 전부 맞는지 확인 하는 용

In [ ]:
# 이미지 파일을 다른 파일로 옮기는데 사용 
# ex) 데이터 프레임에서 이미지 이름들을 가져오고 그 이미지를 옮기면 원래 파일에는 데이터 프레임에 없는 이미지만 남게됨

import os
import shutil

# 필터링된 DataFrame에서 '이미지 이름' 컬럼의 값들을 가져옵니다.
image_names = temp_merge['이미지 이름'].tolist()

# 이미지가 저장될 폴더 이름
output_folder = "filtered_images(2022-2024)"

# 폴더가 존재하지 않으면 생성합니다.
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# movie_poster_all 폴더에서 해당 이미지들을 output_folder로 이동시킵니다.
for image_name in image_names:
    source_path = os.path.join("movie_poster_all", image_name)
    if os.path.exists(source_path):
        shutil.move(source_path, output_folder)

print("이미지 이동이 완료되었습니다.")


In [ ]:
# 이미지 파일을 다른 파일로 복사하는데 사용
import os
import shutil

# 필터링된 DataFrame에서 '이미지 이름' 컬럼의 값들을 가져옵니다.
image_names = null_list

# 이미지가 저장될 폴더 이름
output_folder = "xxxxxxxx"

# 폴더가 존재하지 않으면 생성합니다.
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# movie_poster_all 폴더에서 해당 이미지들을 output_folder로 복사합니다.
for image_name in image_names:
    source_path = os.path.join("movie_poster_all", image_name)
    if os.path.exists(source_path):
        shutil.copy(source_path, output_folder)

print("이미지 복사가 완료되었습니다.")


In [ ]:
# 이미지를 모아 둔 파일 끼리 크로스 체크용

import os

# error_images_all 폴더 내의 모든 jpg 파일 가져오기
error_image_files = [file for file in os.listdir("error_images_all") if file.endswith('.jpg')]

# 압축 이미지 파일 내의 모든 jpg 파일 가져오기
compressed_image_files = [file for file in os.listdir("압축 이미지") if file.endswith('.jpg')]

# 같지 않은 이미지 파일 찾기
different_images = [file for file in error_image_files if file not in compressed_image_files]

print("같지 않은 이미지 파일:")
print(different_images)


In [ ]:
# 파일 불러서 리스트로 저장
import pandas as pd
import numpy as np
import os

directory = r'C:/Users/NT551XCJ/Desktop/MLproject/image_1693제외'

files = os.listdir(directory)

# directory 경로 안에서 jpg인 파일만 파일 이름을 리스트로 저장
jpg_files = [file for file in files if file.endswith('.jpg')]

jpg_files


In [ ]:
# 각 두개의 데이터 프레임에서 이름이 다르거나 없는 행 찾기
# text 데이터의 이미지 이름과 영화 크롤링 데이터 프레임에 있는 이미지 이름이 달라서 아래 코드를 이용해 다른 부분을 찾고 맞게 바꿔서 조인해줌

# kobis_df에서 이미지 이름 추출
df_name = set(text_df['이미지 이름'].unique())

# rgb_df에서 타이틀 추출
text_df_name = set(temp_merge['이미지 이름'].unique())

# kobis_df와 rgb_df에서 각각 다른 이미지 이름을 집합 연산으로 추출
different_names_in_df = df_name - text_df_name
different_names_in_text_df = text_df_name - df_name

# 각각 다른 제목을 리스트로 변환하고 내림차순으로 정렬
different_names_in_df_sorted = sorted(list(different_names_in_df), reverse=True)
different_names_in_text_df_sorted = sorted(list(different_names_in_text_df), reverse=True)

# 결과 출력
print("kobis_df에만 있는 이미지 이름 (내림차순 정렬):")
print(different_names_in_df_sorted)
print("\ntext_df에만 있는 이미지 이름 (내림차순 정렬):")
print(different_names_in_text_df_sorted)


# 포스터에 배우 몇명 출연하는지 분포 확인 (EDA)
분포 확인 용

In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import urllib.request
import urllib
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# 폰트 리스트 출력
import matplotlib.font_manager as fm

font_list = fm.findSystemFonts(fontpaths = None, fontext = 'ttf')
font_list[:]

In [ ]:
# 그래프 한글 오류 해결

from matplotlib import rc
import matplotlib.font_manager as fm
import matplotlib
# 디렉토리 및 파일 이름에 맞추어 변경
font_location = 'C:\\Windows\\Fonts\\malgun.ttf'

font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

matplotlib.rcParams['axes.unicode_minus'] = False

In [ ]:
test.to_csv("face 전체 원본.csv",index = False, encoding = 'cp949')

In [ ]:
# 배우가 없는 애니메이션 같은 값들은 -1을 넣어줌
test = test[['배우','이미지 이름']].fillna(-1)

# 같은 이미지 이름 중 배우 컬럼의 가장 큰 값을 넣어줌 
temp = test.groupby('이미지 이름').max()
temp.reset_index(inplace = True)

# 배우의 명수 분포를 의미하는 데이터프레임
temp = temp.groupby('배우').count()
temp = temp.reset_index()

# 현재 배우가 0부터 시작함 0은 1을 의미함, -1은 0을 의미
# 0이 없음을 뜻하도록 +1을 해줌
temp['배우'] = temp['배우']+1
temp = temp['이미지 이름']

In [ ]:
# 막대 그래프 그리기
plt.figure(figsize=(15, 7))
plt.bar(range(len(temp)), temp, color='orange')
plt.title('배우 수 분포')
plt.xlabel('명수')
plt.ylabel('배우 수')
plt.show()